In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("liverpool-players-analysis").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [7]:
data = spark.read.csv(path="../data/liverpool_players.csv", sep=",", header=True)

In [8]:
data.show(n=10)

+------+--------------------+----------+-----------+-----------+
|    id|                name|  position|dateOfBirth|nationality|
+------+--------------------+----------+-----------+-----------+
|  1795|             Alisson|Goalkeeper| 1992-10-02|     Brazil|
|  8197|              Adrián|Goalkeeper| 1987-01-03|      Spain|
|102046|   Caoimhin Kelleher|Goalkeeper| 1998-11-23|    Ireland|
|  7383|     Kostas Tsimikas|   Defence| 1996-05-12|     Greece|
|  7720|   Sepp van den Berg|   Defence| 2001-12-20|Netherlands|
|  7861|        James Milner|   Defence| 1986-01-04|    England|
|  7862|           Joe Gomez|   Defence| 1997-05-23|    England|
|  7865|          Joël Matip|   Defence| 1991-08-08|   Cameroon|
|  7867|Trent Alexander-A...|   Defence| 1998-10-07|    England|
|  7868|    Andrew Robertson|   Defence| 1994-03-11|   Scotland|
+------+--------------------+----------+-----------+-----------+
only showing top 10 rows

